Calculates discounted cashflows for Fixed-Floating and OIS (TRAINING DATA)

In [87]:
# 1. Imports
import os
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Load your data
#r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima/'
#'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'

# 2. Configuration
your_path = r'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'
train_folder_path = your_path + '6.Active Data/Train Model Data/'
test_folder_path = your_path + '6.Active Data/Test Data/'

# 3. Functions
def convert_to_usd(row,exchange_rates):
    return row['leg1NotionalAmount'] * exchange_rates.get(row['leg1NotionalCurrency'], 1)

def calculate_cashflows(row, period, freq):
    cashflows = []
    start_date = row['effectiveDate']
    end_date = row['expirationDate']
    
    if row[period] == 'MNTH':
        delta = relativedelta(months=row[freq])
    elif row[period] == 'YEAR':
        delta = relativedelta(years=row[freq])
    elif row[period] == 'DAIL':
        delta = timedelta(days=row[freq])
    else:
        raise ValueError(f"Unknown frequency period: {row[period]}")
    
    while start_date <= end_date:
        cashflows.append(start_date)
        start_date += delta

    return cashflows

def convert_to_months(value):
    if "month" in value:
        return int(value.split()[0])
    elif "year" in value:
        return int(value.split()[0]) * 12
    else:
        return 0

def floating_rate(months_to_payout, float_rates_df, bump):
    differences = float_rates_df['MonthsToMaturity'].sub(months_to_payout).abs()
    nearest_index = differences.idxmin()
    return (float_rates_df.loc[nearest_index, 'Rate'] + bump)/ 100


def calculate_discounted_cashflow(row, rate_type, bump, float_rates_df):
    total_discounted_cashflow = 0
    

    # CHECK
    discounted_cashflow_list = []
    cashflow_list = []
    factors = []
    pr_list = []
    drc_list = []
    for date in row['cashflow_dates']:
        
     
        if rate_type == 'fixed':
            time_difference = (date - datetime.now()).days / 365.0 ##### YEARLY BASIS
            cashflow = row['leg1NotionalAmount'] * row['leg1FixedRate']
            discounted_cashflow = cashflow / (1 + row['leg1FixedRate'])**time_difference
            

        if rate_type == 'float':
            if len(cashflow_list) == 0:
                
                time_difference = (date - datetime.now()).days / 365.0
                months_to_payout = time_difference*365 // 30
                payout_rate = floating_rate(months_to_payout,float_rates_df, bump)
                pr_list.append(payout_rate)

                cashflow = row['leg1NotionalAmount'] * payout_rate
                factor = (1 + payout_rate)**time_difference
                factors.append(factor)
                discounted_cashflow = cashflow / factor
                
                
            else:
                time_difference = ((date-row['cashflow_dates'][len(factors)-1])).days/365
                months_to_payout = (date - datetime.now()).days // 30
                payout_rate = floating_rate(months_to_payout,float_rates_df,bump)
                pr_list.append(payout_rate)
                
                cashflow = row['leg1NotionalAmount'] * payout_rate
                prev_factor = factors[-1]
                factor = prev_factor * (1 + payout_rate) ** time_difference
                factors.append(factor)
                discounted_cashflow = cashflow / factor

            
        total_discounted_cashflow += discounted_cashflow
        
        # CHECK
        cashflow_list.append(cashflow)
        discounted_cashflow_list.append(discounted_cashflow)
    
    # CHECK 
    #print('Total discounted CF '+rate_type+': '+str(total_discounted_cashflow))
    #print(rate_type + ' cashflow list ' + str((cashflow_list)))
    #print(rate_type + ' payout rates ' + str((pr_list)))
    #print('\n')
    
    #print(rate_type + ' discounted cashflow list ' + str(discounted_cashflow_list) )
    #print(rate_type + ' discount rate list ' + str((drc_list)))
    #print(rate_type + ' discount factors ' + str((factors)))
    #print('\n')
    return total_discounted_cashflow

def main (folder_path, your_path):
    # 4. Data Loading
    # Load trade data
    files = [f for f in os.listdir(folder_path) if f != '.DS_Store']
    data_file = files[0][:-4]
    data = pd.read_csv(your_path + "2.Cleaned/" + data_file +'_Cleaned.csv')

    # Load exchange rates
    exchange_rates_df = pd.read_csv(your_path + 'exchange_rates.csv')
    exchange_rates = dict(zip(exchange_rates_df['Currency'], exchange_rates_df['Rate_to_USD']))

    # Load floating rates
    euribor_df = pd.read_csv(your_path + 'EURIBOR.csv')
    euribor_df['MonthsToMaturity'] = euribor_df['timeToMaturity'].apply(convert_to_months)
    float_rates_df = euribor_df[['MonthsToMaturity', 'Rate']].copy()
   
    # 5. Data Transformation and Filtering
    filtered_data = data[
        (data['leg1NotionalCurrency'].isin(['EUR', 'USD', 'GBP'])) & 
        (data['leg1UnderlyingAssetOrContractType'].isin(['Fixed-Floating', 'OIS']))
    ]
    df = pd.DataFrame(filtered_data)
    df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
    df['expirationDate'] = pd.to_datetime(df['expirationDate'])

    df['cashflow_dates'] = df.apply(
        lambda row: 
            calculate_cashflows(row, 'leg1FixedRatePaymentFrequencyPeriod', 'leg1FixedRatePaymentFrequencyMultiplier') 
            if row['leg1UnderlyingAssetOrContractType'] == 'Fixed-Floating' 
            else (calculate_cashflows(row, 'leg2UnderlierTenorPeriod', 'leg2UnderlierTenorMultiplier') 
                if row['leg1UnderlyingAssetOrContractType'] == 'OIS' 
                else None), 
        axis=1)

    # 6. Main Operations
    df['leg1NotionalAmountUSD'] = df.apply(lambda row: convert_to_usd(row, exchange_rates), axis=1)
    df['MtM_leg1'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'fixed', 0, float_rates_df), axis=1)
    df['MtM_leg2'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float', 0, float_rates_df), axis=1)
    df['MtM_leg2_bumped'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float', 0.01, float_rates_df), axis=1)
    df['total_delta'] = (df['MtM_leg2'] - df['MtM_leg2_bumped']).abs()
    print(df['cashflow_dates'])
    # 7. Export/Output
    df.drop(columns=['cashflow_dates', 'MtM_leg2_bumped'], inplace=True)
    df.to_csv(your_path + '3.Cash_Risk/' + data_file + '_Cash_Risk.csv', index=False)

main(train_folder_path, your_path)
#main(test_folder_path, your_path)


0    [2022-12-21 00:00:00, 2023-06-21 00:00:00, 202...
Name: cashflow_dates, dtype: object


Now the same for the Test data